# Creating and processing logs

## Goals
  
  - importance of logging
  - configuring logger module
  - open log files
  
## modules

In [ ]:
import logging
import logging.config
import logging.handlers
import mimetypes
import gzip, bz2
import yaml


All automation and scripting activity should be carefully logged.

The ```logging``` module:

  - can stream logs to files and network
  - is configurable via yml 
  - optimizes output via log levels
  - takes care of rotation
  
Hint: manage logs with syslog.
  

In [46]:

# Logs to stdout|err with a default loglevel.
logging.basicConfig(level=logging.DEBUG)

# Create a logger named "mylogger and set its level.
log = logging.getLogger()

# Logs supports a print-like syntax. 
log.info("Use %r instead of %s to avoid", [u"Serialization"], "issues")



In [58]:

%cat logger.yml


version: 1
formatters:
  detailed: 
    class: logging.Formatter
    format: '%(asctime)s %(name)-15s %(levelname)-8s %(processName)-10s %(message)s'

handlers:
  console: 
    class: logging.StreamHandler
    level: INFO
  syslog: 
    class: logging.handlers.SysLogHandler
    formatter: detailed
    level: DEBUG

loggers:
  os:
    handlers: [syslog]

root:
  handlers: [console, syslog]
  level: DEBUG


In [ ]:
with open('logger.yml') as logger_config:
    logger.config.dictConfig(yaml.safe_load(logger_config))
    
# The ```os``` module goes to syslog
log.info("To syslog")


In [59]:
# To process compressed files, use an helper function. 
import mimetools
import gzip
import bz2

def log_open(path):
    """Open log files using its mimetype to choose the correct method"""
    l_type, l_encoding = mimetypes.guess_type(path)
    if l_encoding == 'gzip':
        return gzip.open(path, 'rb')
    elif l_encoding == 'bzip2':
        return bz2.BZ2File(path, 'rb')
    else:
        return open(path, 'rthe b')


## Parsing /proc

Linux /proc filesystem is a cool place to get data

In the next example we'll see how to get:
 - thread informations;
 - disk statistics;
 
 

In [16]:
# Parsing /proc - 1
def linux_threads(pid):
    """Retrieving data from /proc
    """
    from glob import glob
    # glob emulates shell expansion of * and ?
    path = "/proc/{}/task/*/status".format(pid)
    
         
    # pick a set of fields to gather
    t_info = ('Pid', 'Tgid', 'voluntary')  # this is a tuple!
    for t in glob(path):
        # ... and use comprehension to get 
        # intersting data.
        t_info = [x 
                  for x in open(t) 
                  if x.startswith(t_info)] # startswith accepts tuples!
        print(t_info)

In [17]:
# If you're on linux try linux_threads
pid_of_init = 1 # or systemd ?
linux_threads(pid_of_init)

['Tgid:\t1\n', 'Pid:\t1\n', 'voluntary_ctxt_switches:\t446\n']


In [18]:
# On linux /proc/diskstats is the source of I/O infos
disk_l = grep("sda", "/proc/diskstats")
print(''.join(disk_l))

# To gather that data we put the header in a multiline string
from course import diskstats_headers as headers

#Take the 1st entry (sda), split the data...
disk_info = disk_l[0].split()
# ... and tie them with the header
ret = zip(headers, disk_info)

# On py3 we need to iterate over the generators
print(list(ret))

   8       0 sda 216382 11369 8765248 1773709 242383 61222 19926032 6871893 0 783857 8646230
   8       1 sda1 122 0 1856 2097 0 0 0 0 0 2095 2097
   8       2 sda2 54 0 1312 504 0 0 0 0 0 485 504
   8       3 sda3 116 0 1808 1539 0 0 0 0 0 1506 1539
   8       4 sda4 118 0 1824 498 0 0 0 0 0 471 498
   8       5 sda5 267 206 2161 2898 0 0 0 0 0 2441 2898
   8       6 sda6 120 0 1840 979 0 0 0 0 0 958 979
   8       7 sda7 228 1 4442 6227 13 1 112 636 0 5525 6862
   8       8 sda8 212805 9791 8716589 1744819 207751 35245 19690304 6654285 0 619100 8399593
   8       9 sda9 2418 1371 31440 13085 3475 25976 235616 19476 0 27651 32554

[(u'major', '8'), (u'minor', '0'), (u'device', 'sda'), (u'reads', '216382'), (u'reads_merged', '11369'), (u'reads_sectors', '8765248'), (u'reads_ms', '1773709'), (u'writes', '242383'), (u'writes_merged', '61222'), (u'writes_sectors', '19926032'), (u'writes_ms', '6871893'), (u'io_in_progress', '0'), (u'io_ms_spent', '783857'), (u'io_ms_weight', '8646230')]


In [19]:
# Try to mangle ret
print('\n'.join(str(x) for x in ret))

(u'major', '8')
(u'minor', '0')
(u'device', 'sda')
(u'reads', '216382')
(u'reads_merged', '11369')
(u'reads_sectors', '8765248')
(u'reads_ms', '1773709')
(u'writes', '242383')
(u'writes_merged', '61222')
(u'writes_sectors', '19926032')
(u'writes_ms', '6871893')
(u'io_in_progress', '0')
(u'io_ms_spent', '783857')
(u'io_ms_weight', '8646230')


In [20]:
# We can create a reusable commodity class with
from collections import namedtuple

# using the imported `headers` as attributes
# like the one provided by psutil
DiskStats = namedtuple('DiskStat', headers)

# ... and disk_info as values
dstat = DiskStats(*disk_info)
print(dstat.device, dstat.writes_ms)

# Homework: check further features with
# help(collections)

sda 6871893


In [21]:
# Exercise
# Write the following function 
def linux_diskstats(partition):
    """Print every second I/O information from /proc/diskstats
    
        @param: partition - eg sda1 or vdx1
        
        Hint: use the above `grep` function
        Hint: use zip, time.sleep, print() and *magic
    """
    diskstats_headers = ('reads reads_merged reads_sectors reads_ms'
            ' writes writes_merged writes_sectors writes_ms'
            ' io_in_progress io_ms_weight').split()
    
    while True:
        raise NotImplementedError
        print(values, sep="\t")

In [ ]:
# Solution
%load course/linux_diskstats.py

In [24]:
# Using check_output with split() doesn't always work
from os import makedirs
makedirs('/tmp/course/b l a n k s', exist_ok=True)

check_output('ls "/tmp/course/b l a n k s"'.split())

TypeError: makedirs() got an unexpected keyword argument 'exist_ok'

In [30]:
# You can use
from shlex import split
# and
cmd = split('dir -a "/tmp/course/b l a n k s"')
check_output(cmd)

'.  ..\n'

## zip on py3 is a generator 


In [32]:
# zip_iterables():
"""The zip method joins list elements pairwise
    like a zip fastener
"""
from sys import version_info as python_version
a_list = [0, 1, 2, 3]
b_list = ["a", "b", "c", "d"]
zipper = zip(a_list, b_list)
print(zipper)

[(0, u'a'), (1, u'b'), (2, u'c'), (3, u'd')]


In [33]:
if python_version >= (3,):
    zipper = list(zipper)
assert zipper == [(0, "a"), (1, "b"), (2, "c"), (3, "d")]